In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import re

#read in .csv files as df
edge_data = pd.read_csv("edge_us_products_full.csv")
salsify_data = pd.read_csv("salsify_us_products.csv", low_memory = False)

In [2]:
#isolate edge and salsify amazon data
edge_amzn = edge_data[edge_data["upc"] != "Not found in VC"]
salsify_amzn = salsify_data

print(edge_amzn.shape[0])
print(salsify_amzn.shape[0])

print(salsify_amzn["Product status (code)"].unique())

116
44049
[nan 'Active-Purchased Item' 'Active-Stock End Item'
 'Inactive - Use Up Inventory' 'Obsolete']


In [3]:
print(edge_amzn.columns)

Index(['Unnamed: 0', 'retailer', 'rpc', 'upc', 'from Salsify', 'Unnamed: 5',
       'mpc', 'product_description', 'brand', 'category', 'min_price',
       'max_price', 'status', 'date_modified', 'last_modified_by',
       'Hierarchy Mid Level', 'Segment', 'Hierarchy Lowest Level',
       'Priority SKU'],
      dtype='object')


In [4]:
print(salsify_amzn.columns)

Index(['Entity ID', 'Product Sequencing L1', 'Target Market(s)',
       'Target Market(s).1', 'BIC Item Name', 'Product Sequencing L1 Code',
       'Local Code', 'Product Sequencing L2', 'Product Sequencing L2 Code',
       'Customer',
       ...
       'Hazardous ingredients (CLP)', 'Sales Packaging',
       'SEQ 5 - Environmental certification - NF 400',
       'SEQ 5 - Environmental declaration - % Recycled Material',
       'Signal word (CLP)', 'Stocking Type-en-GB', 'test prop - en-GB',
       'salsify:parent_id', 'salsify:data_inheritance_hierarchy_level_id',
       'salsify:profile_asset_id'],
      dtype='object', length=1293)


In [5]:
#get UPCs from edge data
edge_upc_list = edge_amzn["upc"].unique()
print(len(edge_upc_list))
print(edge_amzn[edge_amzn["upc"].isnull()].shape[0])
print("\n")

#85 unique, 51 null, 18 duplicates
print(edge_amzn["upc"].value_counts().head())
print("\n")

#identify where dupblicates are
dup_upc_list = edge_amzn["upc"].value_counts().head(3).index
print(edge_amzn.loc[edge_amzn["upc"].isin(dup_upc_list), "brand"].unique())
print("\n")

#UPC 70330185104 has two entries in edge data

#130 entries, 81 unique UPCs, 50 null

115
0


70330185104    2
70330722620    1
70330411715    1
70330522909    1
70330537224    1
Name: upc, dtype: int64


['Velocity' 'Comfort 3 Hybrid']




In [6]:
for num in range(1,8):
    print(edge_amzn["upc"].unique()[num])
    
#salsify_amzn.to_csv("salsify_amzn_us_products.csv")


70330185104
70330362055
70330312579
70330341289
70330346857
70330655508
70330730274


In [7]:
pd.options.mode.chained_assignment = None

#goal - end up with column of UPCs for salsify_amzn
#look for UPC in numeric columns, as part of text columns

#numeric columns - "EAN 13 - UPC for Sales Packaging", "EAN-UPC for Inner Case"
#string columns - "Child GTINs", "Child GTINs.1", "Child GTINs.2", "PACKAGES"

#additional columns - "Case GTIN", "EAN 14 -UPC for Outer Case", "Inner GTIN"

#create new dataframe with index, value for each of these columns
upc_num_col = ["EAN 13 - UPC for Sales Packaging","EAN-UPC for Inner Case","Inner GTIN","EAN 14 -UPC for Outer Case"]
upc_num_col.append("Case GTIN")
salsify_upc = salsify_amzn.loc[:,upc_num_col]
#print(salsify_upc.head())

for col in salsify_amzn.columns:
    if re.search("PACKAGES",col):
        print(col)

PACKAGES
PACKAGES.1
PACKAGES.2
PACKAGES.3


In [8]:
#print(salsify_amzn[["Child GTINs", "Child GTINs.1", "Child GTINs.2", "PACKAGES"]])
list_str = ["Child GTINs", "Child GTINs.1", "Child GTINs.2", "PACKAGES", "PACKAGES.1", "PACKAGES.2", "PACKAGES.3"]
for key in list_str:
    salsify_upc[key] = salsify_amzn[key].str.split("_").str[-1].astype(float)
    
salsify_upc = salsify_upc[salsify_upc.notnull().sum(axis=1) > 0]
#print(salsify_upc)
print(salsify_upc.shape[0])
print("\n")

#salsify_upc.to_csv("upc_list.csv")
#narrowed down from 44049 to 9346

print(salsify_upc.isnull().sum())
print("\n")

for column in salsify_upc.columns:
    print(len(salsify_upc[column].unique()))

#salsify_upc.to_csv("upc_list_2")
print(salsify_upc.columns)

24122


EAN 13 - UPC for Sales Packaging    16329
EAN-UPC for Inner Case              17944
Inner GTIN                          17947
EAN 14 -UPC for Outer Case          15516
Case GTIN                           15527
Child GTINs                         14911
Child GTINs.1                       17089
Child GTINs.2                       20784
PACKAGES                            14939
PACKAGES.1                          17125
PACKAGES.2                          21572
PACKAGES.3                          24120
dtype: int64


3470
3420
3431
5284
5277
5613
4008
1245
4678
4298
1170
3
Index(['EAN 13 - UPC for Sales Packaging', 'EAN-UPC for Inner Case',
       'Inner GTIN', 'EAN 14 -UPC for Outer Case', 'Case GTIN', 'Child GTINs',
       'Child GTINs.1', 'Child GTINs.2', 'PACKAGES', 'PACKAGES.1',
       'PACKAGES.2', 'PACKAGES.3'],
      dtype='object')


In [9]:
#for each row, loop through each column, check if in edge_upc_list, assign code as upc for row, set boolean
edge_upc_list = edge_upc_list.tolist()
edge_upc_list = [int(x) for x in edge_upc_list]
edge_upc_list = [str(x) for x in edge_upc_list]

print(edge_upc_list)
print(salsify_upc.columns)


['70330145405', '70330185104', '70330362055', '70330312579', '70330341289', '70330346857', '70330655508', '70330730274', '70330739789', '70330682719', '70330189867', '70330189874', '70330195141', '70330730281', '70330740945', '70330361591', '70330522909', '70330522916', '70330358157', '70330355651', '70330534117', '70330534124', '70330534094', '70330534100', '70330351561', '70330432109', '70330188389', '70330730809', '70330437425', '70330437432', '70330437364', '70330196490', '70330176331', '70330532137', '70330524873', '70330131620', '70330531390', '70330524866', '70330531383', '70330181304', '70330739772', '70330123465', '70330196506', '70330719873', '70330748262', '70330747845', '70330748279', '70330743274', '70330144088', '70330522121', '70330411715', '70330507906', '70330530577', '70330529793', '70330505896', '70330508156', '70330905764', '70330417557', '70330426566', '70330433649', '70330406964', '70330417144', '70330362086', '70330371422', '70330722620', '70330743489', '70330743

In [10]:
print(salsify_upc.shape[0])
print(salsify_upc.columns)

24122
Index(['EAN 13 - UPC for Sales Packaging', 'EAN-UPC for Inner Case',
       'Inner GTIN', 'EAN 14 -UPC for Outer Case', 'Case GTIN', 'Child GTINs',
       'Child GTINs.1', 'Child GTINs.2', 'PACKAGES', 'PACKAGES.1',
       'PACKAGES.2', 'PACKAGES.3'],
      dtype='object')


In [11]:
counter = 0

def row_match(row):
    global counter
    row = row.astype(str)
    pattern = '|'.join(edge_upc_list)
    pattern = "(" + pattern + ")"
    match_series = row.str.extract(pattern)
    match_series = match_series.dropna()
    if match_series.empty == False:
        counter += 1
        return match_series.iloc[0,0]

salsify_upc["upc"] = salsify_upc.apply(row_match,axis=1)

print(salsify_upc["upc"].notnull().sum())
print(counter)

#print(salsify_upc.head())

620
620


In [12]:
for code in edge_upc_list:
    if code not in salsify_upc["upc"].unique():
        print(code)
        
print(len(salsify_upc["upc"].unique()))

print(salsify_upc.columns)

salsify_upc["bool"] = salsify_upc["upc"].notnull()

#use PACKAGES column
salsify_upc["upc_2"] = salsify_upc["PACKAGES"]
salsify_upc["exist_bool"] = salsify_upc["upc"].isin(edge_upc_list)

print(salsify_upc[salsify_upc["exist_bool"] == True].shape[0])
print(salsify_upc[salsify_upc["bool"] == True].shape[0])

#salsify_upc.to_csv("salsify_upc6.csv")

116
Index(['EAN 13 - UPC for Sales Packaging', 'EAN-UPC for Inner Case',
       'Inner GTIN', 'EAN 14 -UPC for Outer Case', 'Case GTIN', 'Child GTINs',
       'Child GTINs.1', 'Child GTINs.2', 'PACKAGES', 'PACKAGES.1',
       'PACKAGES.2', 'PACKAGES.3', 'upc'],
      dtype='object')
620
620


In [13]:
pattern = '|'.join(edge_upc_list)
pattern = "(" + pattern + ")"
print(pattern)

print(len(edge_upc_list))

(70330145405|70330185104|70330362055|70330312579|70330341289|70330346857|70330655508|70330730274|70330739789|70330682719|70330189867|70330189874|70330195141|70330730281|70330740945|70330361591|70330522909|70330522916|70330358157|70330355651|70330534117|70330534124|70330534094|70330534100|70330351561|70330432109|70330188389|70330730809|70330437425|70330437432|70330437364|70330196490|70330176331|70330532137|70330524873|70330131620|70330531390|70330524866|70330531383|70330181304|70330739772|70330123465|70330196506|70330719873|70330748262|70330747845|70330748279|70330743274|70330144088|70330522121|70330411715|70330507906|70330530577|70330529793|70330505896|70330508156|70330905764|70330417557|70330426566|70330433649|70330406964|70330417144|70330362086|70330371422|70330722620|70330743489|70330743465|70330743472|70330743496|70330743595|70330743588|70330743571|70330745919|70330723399|70330246232|70330366046|70330364660|70330742192|70330196513|70330343610|70330143470|70330375291|70330338999|703

In [14]:
salsify_amzn["upc"] = salsify_upc["upc"]
salsify_amzn["bool"] = salsify_upc["bool"]

salsify_amzn_true = salsify_amzn[salsify_amzn["bool"] == True]
#salsify_amzn_true.to_csv("salsify_true3.csv")

In [15]:
print(salsify_amzn_true.shape[0])

620


In [19]:
code_list = salsify_amzn_true["upc"].unique()
print(len(code_list))
freq_bool = salsify_amzn_true["upc"].value_counts().gt(1)

dup_codes = salsify_amzn_true["upc"].value_counts()[freq_bool].index

#salsify_amzn_true.to_csv("salsify_amzn_true.csv")

for code in dup_codes:
    temp_df = salsify_amzn_true[salsify_amzn_true["upc"] == code]
    
    #remove inactive
    if temp_df["Product status (code)"].str.contains("Active").any():
        temp_df = temp_df[temp_df["Product status (code)"].str.split("-").str[0].str.strip() == "Active"]
        
    #find latest begin ship date
    if temp_df["Begin Ship Date - en-US"].notnull().any():
        temp_df["Begin Ship Date - en-US"] = pd.to_datetime(temp_df["Begin Ship Date - en-US"])
        temp_df = temp_df[temp_df["Begin Ship Date - en-US"] == temp_df["Begin Ship Date - en-US"].max()]
        temp_df["Begin Ship Date - en-US"] = temp_df["Begin Ship Date - en-US"].dt.strftime("%m/%d/%Y")

    #find latest PIM SKU Creation Date
    if temp_df["PIM SKU Creation Date"].notnull().any():
        temp_df["PIM SKU Creation Date"] = pd.to_datetime(temp_df["PIM SKU Creation Date"])
        temp_df = temp_df[temp_df["PIM SKU Creation Date"] == temp_df["PIM SKU Creation Date"].max()]
        temp_df["PIM SKU Creation Date"] = temp_df["PIM SKU Creation Date"].dt.strftime("%m/%d/%Y")
    
    #find latest end ship date
    if temp_df["End Ship Date - en-US"].notnull().any():
        temp_df["End Ship Date - en-US"] = pd.to_datetime(temp_df["End Ship Date - en-US"])
        temp_df = temp_df[temp_df["End Ship Date - en-US"] == temp_df["End Ship Date - en-US"]]
        temp_df["End Ship Date - en-US"] = temp_df["End Ship Date - en-US"].dt.strftime("%m/%d/%Y")

    salsify_amzn_true = salsify_amzn_true[salsify_amzn_true["upc"] != code]
    salsify_amzn_true = pd.concat([salsify_amzn_true, temp_df])
    
salsify_amzn_true.to_csv("salsify_amzn_final.csv")

115


In [18]:
print(salsify_amzn_true.shape[0])

115
